<a href="https://colab.research.google.com/github/Caoyq1992/SMRT/blob/master/%E8%AE%AF%E9%A3%9E%E6%94%B9%E5%86%99.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# -*- coding:utf-8 -*-
from datetime import datetime
from wsgiref.handlers import format_date_time
from time import mktime
import hashlib
import base64
import hmac
from urllib.parse import urlencode
import json
import requests

# **段落改写** 

In [5]:
#@title 改写等级 L1  ~  L6  等级越高，改写程度越深
class AssembleHeaderException(Exception):
    def __init__(self, msg):
        self.message = msg

class Url:
    def __init__(this, host, path, schema):
        this.host = host
        this.path = path
        this.schema = schema
        pass

class wsParam(object):
    def __init__(self):
        self.APPID = APPID
        self.APIKey = APIKey
        self.APISecret = APISecret
        self.url = 'https://api.xf-yun.com/v1/private/se3acbe7f'
        self.level = level

    def parse_url(self,requset_url):
        stidx = requset_url.index("://")
        host = requset_url[stidx + 3:]
        schema = requset_url[:stidx + 3]
        edidx = host.index("/")
        if edidx <= 0:
            raise AssembleHeaderException("invalid request url:" + requset_url)
        path = host[edidx:]
        host = host[:edidx]
        u = Url(host, path, schema)
        return u

    def init_header(self):
        headers = {
            'content-type': "application/json",
            'host': 'api.xf-yun.com'
        }
        return headers

    def get_body(self):
        data = {
            "header": {
                "app_id": self.APPID,
                "status": 3,
            },
            "parameter": {
                "se3acbe7f": {
                    "level": self.level,
                    "result": {
                        "encoding": "utf8",
                        "compress": "raw",
                        "format": "json"
                    }
                }
            },
            "payload": {
                "input1": {
                    "encoding": "utf8",
                    "compress": "raw",
                    "format": "plain",
                    "status": 3,
                    "text": str(base64.b64encode(text.encode('utf-8')), 'utf-8')
                }
            }
        }
        body = json.dumps(data)
        return body

def assemble_ws_auth_url(requset_url, method="POST", api_key="", api_secret=""):
    u = wsParam.parse_url(requset_url)
    host = u.host
    path = u.path
    now = datetime.now()
    date = format_date_time(mktime(now.timetuple()))
    print(date)
    # date = "Thu, 12 Dec 2019 01:57:27 GMT"
    signature_origin = "host: {}\ndate: {}\n{} {} HTTP/1.1".format(host, date, method, path)
    print("----2",signature_origin)
    signature_sha = hmac.new(api_secret.encode('utf-8'), signature_origin.encode('utf-8'),
                             digestmod=hashlib.sha256).digest()
    signature_sha = base64.b64encode(signature_sha).decode(encoding='utf-8')
    authorization_origin = "api_key=\"%s\", algorithm=\"%s\", headers=\"%s\", signature=\"%s\"" % (
        api_key, "hmac-sha256", "host date request-line", signature_sha)
    print("----1:",authorization_origin)
    authorization = base64.b64encode(authorization_origin.encode('utf-8')).decode(encoding='utf-8')
    print(authorization_origin)
    values = {
        "host": host,
        "date": date,
        "authorization": authorization
    }
    return requset_url + "?" + urlencode(values)

def get_result():
    request_url = assemble_ws_auth_url(wsParam.url, "POST", wsParam.APIKey, wsParam.APISecret)
    print("request_url:", request_url)
    response = requests.post(request_url, data=wsParam.get_body(), headers=wsParam.init_header())
    print("response:", response)
    str_result = response.content.decode('utf8')
    json_result = json.loads(str_result)
    print("response-content:", json_result)
    if json_result. __contains__('header') and json_result['header']['code'] == 0:
        renew_text = json_result['payload']['result']['text']
        renew_word_text = renew_text[0]
        print("\n改写结果：", str(base64.b64decode(renew_text), 'utf-8'))
        #改写结果保存到文件
        #result_file = open(".\改写结果.txt",'w',encoding='utf-8')
        #result_file.write(str(base64.b64decode(renew_text), 'utf-8'))


if __name__ == "__main__":
    APPID = "4cf9af93" #@param {type:"string"}
    APISecret = "YmQ1YjVhMWMxNGM4YTE5ZDFkYjg5YmIz" #@param {type:"string"}
    APIKey = "17931160ace4e60d57c560ad2457b0e2" #@param {type:"string"}
    level = "<L6>" #@param {type:"string"}#改写等级 <L1>  ~  <L6>  等级越高，改写程度越深
    text = "\u968F\u7740\u6211\u56FD\u57CE\u5E02\u5316\u811A\u6B65\u7684\u4E0D\u65AD\u52A0\u5FEB\uFF0C\u56ED\u6797\u5DE5\u7A0B\u5EFA\u8BBE\u7684\u6570\u91CF\u4E5F\u5728\u4E0D\u65AD\u4E0A\u5347\uFF0C\u57CE\u5E02\u5BF9\u4E8E\u56ED\u6797\u5DE5\u7A0B\u7684\u8D28\u91CF\u8981\u6C42\u4E5F\u968F\u4E4B\u4E0A\u5347\uFF0C\u7136\u800C\u5C31\u5F53\u524D\u6211\u56FD\u56ED." #@param {type:"string"}
#@markdown 

    # 上传文件
    #text = open(r"文本.txt",encoding='utf-8').read()
    wsParam = wsParam()
    get_result()

Fri, 07 Oct 2022 04:41:46 GMT
----2 host: api.xf-yun.com
date: Fri, 07 Oct 2022 04:41:46 GMT
POST /v1/private/se3acbe7f HTTP/1.1
----1: api_key="17931160ace4e60d57c560ad2457b0e2", algorithm="hmac-sha256", headers="host date request-line", signature="QWM0cZHRnS96I4SV0rfRFVenVk6qh0vDHGWXrTtjUYY="
api_key="17931160ace4e60d57c560ad2457b0e2", algorithm="hmac-sha256", headers="host date request-line", signature="QWM0cZHRnS96I4SV0rfRFVenVk6qh0vDHGWXrTtjUYY="
request_url: https://api.xf-yun.com/v1/private/se3acbe7f?host=api.xf-yun.com&date=Fri%2C+07+Oct+2022+04%3A41%3A46+GMT&authorization=YXBpX2tleT0iMTc5MzExNjBhY2U0ZTYwZDU3YzU2MGFkMjQ1N2IwZTIiLCBhbGdvcml0aG09ImhtYWMtc2hhMjU2IiwgaGVhZGVycz0iaG9zdCBkYXRlIHJlcXVlc3QtbGluZSIsIHNpZ25hdHVyZT0iUVdNMGNaSFJuUzk2STRTVjByZlJGVmVuVms2cWgwdkRIR1dYclR0alVZWT0i


ConnectionError: ignored

# **文档改写**

In [ ]:
#@title 改写等级 L1 ~ L6 等级越高，改写程度越深
class AssembleHeaderException(Exception):
    def __init__(self, msg):
        self.message = msg

class Url:
    def __init__(this, host, path, schema):
        this.host = host
        this.path = path
        this.schema = schema
        pass

class wsParam(object):
    def __init__(self):
        self.APPID = APPID
        self.APIKey = APIKey
        self.APISecret = APISecret
        self.url = 'https://api.xf-yun.com/v1/private/se3acbe7f'
        self.level = level

    def parse_url(self,requset_url):
        stidx = requset_url.index("://")
        host = requset_url[stidx + 3:]
        schema = requset_url[:stidx + 3]
        edidx = host.index("/")
        if edidx <= 0:
            raise AssembleHeaderException("invalid request url:" + requset_url)
        path = host[edidx:]
        host = host[:edidx]
        u = Url(host, path, schema)
        return u

    def init_header(self):
        headers = {
            'content-type': "application/json",
            'host': 'api.xf-yun.com'
        }
        return headers

    def get_body(self):
        data = {
            "header": {
                "app_id": self.APPID,
                "status": 3,
            },
            "parameter": {
                "se3acbe7f": {
                    "level": self.level,
                    "result": {
                        "encoding": "utf8",
                        "compress": "raw",
                        "format": "json"
                    }
                }
            },
            "payload": {
                "input1": {
                    "encoding": "utf8",
                    "compress": "raw",
                    "format": "plain",
                    "status": 3,
                    "text": str(base64.b64encode(text.encode('utf-8')), 'utf-8')
                }
            }
        }
        body = json.dumps(data)
        return body

def assemble_ws_auth_url(requset_url, method="POST", api_key="", api_secret=""):
    u = wsParam.parse_url(requset_url)
    host = u.host
    path = u.path
    now = datetime.now()
    date = format_date_time(mktime(now.timetuple()))
    print(date)
    # date = "Thu, 12 Dec 2019 01:57:27 GMT"
    signature_origin = "host: {}\ndate: {}\n{} {} HTTP/1.1".format(host, date, method, path)
    print("----2",signature_origin)
    signature_sha = hmac.new(api_secret.encode('utf-8'), signature_origin.encode('utf-8'),
                             digestmod=hashlib.sha256).digest()
    signature_sha = base64.b64encode(signature_sha).decode(encoding='utf-8')
    authorization_origin = "api_key=\"%s\", algorithm=\"%s\", headers=\"%s\", signature=\"%s\"" % (
        api_key, "hmac-sha256", "host date request-line", signature_sha)
    print("----1:",authorization_origin)
    authorization = base64.b64encode(authorization_origin.encode('utf-8')).decode(encoding='utf-8')
    print(authorization_origin)
    values = {
        "host": host,
        "date": date,
        "authorization": authorization
    }
    return requset_url + "?" + urlencode(values)

def get_result():
    request_url = assemble_ws_auth_url(wsParam.url, "POST", wsParam.APIKey, wsParam.APISecret)
    print("request_url:", request_url)
    response = requests.post(request_url, data=wsParam.get_body(), headers=wsParam.init_header())
    print("response:", response)
    str_result = response.content.decode('utf8')
    json_result = json.loads(str_result)
    print("response-content:", json_result)
    if json_result. __contains__('header') and json_result['header']['code'] == 0:
        renew_text = json_result['payload']['result']['text']
        print("\n改写结果：", str(base64.b64decode(renew_text), 'utf-8'))
        #改写结果保存到文件
        result_file = open("/content/改写结果.txt",'w',encoding='utf-8')
        result_file.write(str(base64.b64decode(renew_text), 'utf-8'))
        print('1')


if __name__ == "__main__":
    APPID = "4cf9af93" #@param {type:"string"}
    APISecret = "YmQ1YjVhMWMxNGM4YTE5ZDFkYjg5YmIz" #@param {type:"string"}
    APIKey = "17931160ace4e60d57c560ad2457b0e2" #@param {type:"string"}
    level = "<L6>" #@param {type:"string"}#改写等级 <L1>  ~  <L6>  等级越高，改写程度越深
    # 上传文件
    text_dir = "/content/\u65B0\u5EFA Microsoft Word \u6587\u6863.docx" #@param {type:"string"}
    text = open(text_dir,encoding='utf-8').read()
    wsParam = wsParam()
    get_result()

UnicodeDecodeError: ignored